In [ ]:
import os
import sys
import argparse
import pandas as pd
import torch
import anndata as ad
from tqdm import tqdm

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from DeepRUOT.losses import OT_loss1
from DeepRUOT.utils import (
    generate_steps, load_and_merge_config,
    SchrodingerBridgeConditionalFlowMatcher,
    generate_state_trajectory, get_batch, get_batch_size
)
from DeepRUOT.train import train_un1_reduce, train_all
from DeepRUOT.models import FNet, scoreNet2
from DeepRUOT.constants import DATA_DIR, RES_DIR
from DeepRUOT.exp import setup_exp

### Load config

In [ ]:
config_path = '/lustre/home/2100011778/DeepRUOTv2_test_data/config/test_data.yaml'

# Load and merge configuration
config = load_and_merge_config(config_path)

### Load data and model

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, config['data']['file_path']))
df = df.iloc[:, :config['data']['dim'] + 1]
device = torch.device('cpu')
exp_dir, logger = setup_exp(
            RES_DIR, 
            config, 
            config['exp']['name']
        )
dim = config['data']['dim']

In [ ]:
model_config = config['model']
        
f_net = FNet(
    in_out_dim=model_config['in_out_dim'],
    hidden_dim=model_config['hidden_dim'],
    n_hiddens=model_config['n_hiddens'],
    activation=model_config['activation']
).to(device)

sf2m_score_model = scoreNet2(
    in_out_dim=model_config['in_out_dim'],
    hidden_dim=model_config['score_hidden_dim'],
    activation=model_config['activation']
).float().to(device)

In [ ]:
f_net.load_state_dict(torch.load(os.path.join(exp_dir, 'model_final'),map_location=torch.device('cpu')))
f_net.to(device)
sf2m_score_model.load_state_dict(torch.load(os.path.join(exp_dir, 'score_model_final'),map_location=torch.device('cpu')))
sf2m_score_model.to(device)

### Dim reduction (Optional)

In [ ]:
import numpy as np
import joblib
from sklearn.decomposition import PCA  # Import PCA
import matplotlib.pyplot as plt
import seaborn as sns

import umap
# umap_op = PCA(n_components=2)
umap_op = umap.UMAP(n_components=2, random_state=42) # You may change UMAP to PCA or other dimension reduction methods
xu = umap_op.fit_transform(df.iloc[:, 1:])  # Assuming df is your DataFrame
joblib.dump(umap_op, os.path.join(exp_dir, 'dim_reduction.pkl'))  # Save the UMAP model

### Plot Velocity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import joblib
# Assume the following are defined:
# - df: DataFrame with columns 'samples', 'x1', 'x2', ..., 'xn'
# - dim: Number of dimensions (n)
# - device: torch.device (e.g., 'cpu' or 'cpu')
# - sf2m_score_model: Your model function that takes t_tensor and data_tensor

dim_reducer = joblib.load(os.path.join(exp_dir, 'dim_reduction.pkl')) # If no dim reducer is needed, set this to None

# Step 1: Extract all time points and data points


all_times = df['samples'].values
all_data = df[[f'x{i}' for i in range(1, dim + 1)]].values

#t_value = 2.0  
t_tensor = torch.tensor(all_times).unsqueeze(1).float().to(device)

# Step 2: Convert to PyTorch tensors
data_tensor = torch.tensor(all_data, dtype=torch.float32).to(device)
data_tensor.requires_grad_(True)  # Enable gradient tracking
#t_tensor = torch.tensor(all_times, dtype=torch.float32).unsqueeze(1).to(device)

lnw0 = torch.log(torch.ones(data_tensor.shape[0], 1) / data_tensor.shape[0]).to(device)
# Step 3: Compute log density values and gradients
gradients = f_net.v_net(t_tensor, data_tensor) 

# Step 4: Move data to CPU for plotting
data_np = data_tensor.detach().cpu().numpy()
if dim_reducer is not None:
    data_np_2d=dim_reducer.transform(data_np)
else:
    data_np_2d=data_np[:,:2]
gradients_np = gradients.detach().cpu().numpy()
data_end = data_np + gradients_np
if dim_reducer is not None:
    data_end_2d=dim_reducer.transform(data_end)
else:
    data_end_2d=data_end[:,:2]
gradients_np=data_end_2d - data_np_2d

gradients_np = gradients_np / np.linalg.norm(gradients_np, axis = 1, keepdims=True) * 5
times_np = all_times
data_np = data_np_2d

In [ ]:
import anndata
import scvelo as scv
import scanpy as sc
import numpy as np

# Assume your data is already defined
# all_data: 50-dimensional data with shape (n_cells, 50)
# gradients: 50-dimensional vector field with shape (n_cells, 50) 
# pca: Trained PCA or UMAP model for generating X_umap
# X_umap: Pre-computed UMAP embeddings with shape (n_cells, 2)

# Create AnnData object
adata = anndata.AnnData(X=all_data)

# Set 'Ms' layer to avoid KeyError: 'Ms'
adata.layers['Ms'] = all_data  # Use original 50-dim data as state matrix
# Optional: normalize and log transform (based on data needs)
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)
# adata.layers['Ms'] = adata.X.copy()

# Set velocity vectors
adata.layers['velocity'] = gradients.detach().cpu().numpy()  # Store velocity vectors in layers
print(adata.layers['velocity'].shape)  # Confirm shape


sc.pp.neighbors(adata, use_rep='X', n_neighbors=30, metric='euclidean', random_state=42)
# 计算 UMAP
sc.tl.umap(adata)

# Set pre-computed UMAP embeddings
if dim_reducer is not None:
    X_umap = dim_reducer.transform(all_data)  # Assume pca is trained dim reduction model
else:
    X_umap = all_data[:,:2]
# adata.obsm['X_umap'] = X_umap
print(adata.obsm['X_umap'].shape)  # Confirm UMAP embedding shape
adata.obs['time'] = all_times
if adata.layers['velocity'].shape[1] != 2:
    # Calculate neighbor graph (required for velocity graph)
    sc.pp.neighbors(adata, n_neighbors=30, use_rep='X')  # Calculate neighbors based on high-dim data

    # Calculate velocity graph
    scv.tl.velocity_graph(adata, vkey='velocity', n_jobs=16)  # Build velocity graph from high-dim velocity vectors

    # Project velocities to UMAP space
    scv.tl.velocity_embedding(adata, basis='umap', vkey='velocity')  # Project velocities to UMAP
else:
    adata.obsm['velocity_umap'] = adata.layers['velocity']

# Plot

adata.obs['time_categorical'] = pd.Categorical(adata.obs['time'])


In [ ]:
import scanpy as sc
meta_data = sc.read_h5ad('/lustre/home/2100011778/spatial_data/h5ad/stromal_mnn.h5ad')
# meta_data = pd.read_csv('D:\DeepRUOTv2\\test_data\data_0625\WO_FG_D10_D20_pro_HB_tumor_metadata.csv', sep=' ')
# Split celltype by underscore and take everything except first word
# meta_data['celltype'] = meta_data['celltype'].str.split('_').str[1:].str.join('_')
print(meta_data.obs['cellType_new3'])
adata.obs['celltype'] = meta_data.obs['cellType_new3'].values

# Visualization settings
scv.settings.set_figure_params('scvelo')  # Set scvelo plotting style

In [ ]:
meta_data

In [ ]:
meta_data.obsm['X_pca'] = df.iloc[:,1:].values
#meta_data.layers['velocity'] = gradients.detach().cpu().numpy()
# Calculate velocity graph
#scv.tl.velocity_graph(meta_data, vkey='velocity', n_jobs=16)  # Build velocity graph from high-dim velocity vectors

# Project velocities to UMAP space
#scv.tl.velocity_embedding(meta_data, basis='umap', vkey='velocity')  # Project velocities to UMAP
sc.pp.neighbors(meta_data, n_neighbors=30, use_rep='X_pca')
sc.tl.paga(meta_data, groups='celltype')

In [ ]:
adata

In [ ]:
# 计算PAGA图
sc.tl.paga(meta_data, groups='celltype', use_rna_velocity=True)
# scv.tl.paga(meta_data, groups='celltype')

In [ ]:
# 可视化PAGA + Velocity箭头
sc.pl.paga(meta_data, color='celltype', show=False)
# scv.pl.velocity_embedding_stream(adata, basis='umap', color='celltype', ax=plt.gca(), show=False)
# plt.show()

In [ ]:
sc.pl.paga(
  meta_data,
  threshold = 0.15,
  arrowsize = 10,
  edge_width_scale = 0.5,
  transitions = "transitions_confidence",
  dashed_edges = "connectivities",
)

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

# 假设 adata 已经包含PAGA结果和聚类信息（例如 'leiden'）
# 如果没有UMAP嵌入，计算UMAP
sc.tl.umap(meta_data, init_pos='paga')  # 使用PAGA初始化UMAP（可选）

# 绘制UMAP图，按聚类着色
sc.pl.umap(meta_data, color='celltype', title='UMAP colored by Leiden clusters', show=True)
scv.pl.velocity_embedding_stream(meta_data, basis='umap', color='celltype', ax=plt.gca(), show=False)
# # 可选：叠加PAGA路径到UMAP图
# sc.pl.umap(adata, color='celltype', title='UMAP with PAGA paths', show=False)
# sc.pl.paga(adata, plot=True, ax=plt.gca(), show=False)  # 叠加PAGA路径
# plt.show()

# 可选：按特定基因着色
# sc.pl.umap(adata, color=['gene_name1', 'gene_name2'], title='UMAP colored by gene expression')

In [ ]:
adata

In [ ]:
# 使用PAGA初始化的UMAP
sc.tl.umap(adata, init_pos='paga')

# 可视化UMAP图，并可以同时展示PAGA的连接关系
sc.pl.umap(adata,  legend_loc='on data', title='', frameon=False, legend_fontsize=10)
sc.pl.paga_compare(adata, legend_fontsize=10, frameon=False, size=20, title='', legend_fontoutline=2, show=False)

In [ ]:
scv.pl.velocity_embedding_stream(
    adata,
    basis='umap',
    color='celltype',
    figsize=(7, 5),
    density=3,
    title='Velocity Stream Plot',
    legend_loc=None, #'right',
    # Available palettes:
    # - 'tab20': 20 distinct colors
    # - 'viridis': Sequential colormap from dark blue to yellow
    # - 'plasma': Sequential colormap from dark purple to yellow
    # - 'inferno': Sequential colormap from black to yellow
    # - 'magma': Sequential colormap from black to light pink
    # - 'Set1', 'Set2', 'Set3': Qualitative colormaps with distinct colors
    # - 'Paired': Qualitative colormap with paired colors
    # - 'husl': Evenly spaced colors in HUSL space
    palette='tab20',
    save= exp_dir+'/velocity_stream_plot.svg'
)


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import joblib

# Load dimension reducer
dim_reducer = joblib.load(os.path.join(exp_dir, 'dim_reduction.pkl')) # If no dim reducer is needed, set this to None
device = 'cpu'
f_net = f_net.to(device)

def plot_g_values(df, f_net, dim_reducer=None, device=device):
    # Get all time points
    time_points = df['samples'].unique()
    
    # Store data for each time point
    data_by_time = {}
    
    # Calculate g_values for each time point
    for time in time_points:
        subset = df[df['samples'] == time]
        n = dim  # Make sure dim is defined

        # Generate column names
        column_names = [f'x{i}' for i in range(1, n + 1)]

        # Convert each column to tensor and move to device
        tensors = [torch.tensor(subset[col].values, dtype=torch.float32).to(device) for col in column_names]

        # Stack tensors into 2D tensor
        data = torch.stack(tensors, dim=1)
        with torch.no_grad():
            t = torch.tensor([time], dtype=torch.float32).to(device)
            _, g, _, _ = f_net(t, data)
        
        data_by_time[time] = {'data': subset, 'g_values': g.detach().cpu().numpy()}
    
    # Combine all g_values
    all_g_values = np.concatenate([content['g_values'] for content in data_by_time.values()])
    
    # Calculate 95th percentile of g_values
    vmax_value = np.percentile(all_g_values, 95)
    
    # Initialize color mapper with clipping
    norm = plt.Normalize(vmin=0, vmax=vmax_value, clip=True)
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot data for each time point on same axis
    for time, content in data_by_time.items():
        subset = content['data']
        g_values = content['g_values']
        n = dim

        column_names = [f'x{i}' for i in range(1, n + 1)]
        new_data = subset[column_names]
        
        if dim_reducer is not None:
            data_reduced = dim_reducer.transform(new_data)
        else:
            data_reduced = new_data.iloc[:, :2].values
            
        x = adata[df['samples'] == time].obsm['X_umap'][:, 0]#data_reduced[:, 0]
        y = adata[df['samples'] == time].obsm['X_umap'][:, 1] #data_reduced[:, 1]
        
        # Map g_values to colors
        colors = plt.cm.rainbow(norm(g_values))
        
        # Plot scatter with labels for legend
        ax.scatter(x, y, c=colors, label=f'Time {time}', alpha=0.7, marker='o')
    
    ax.set_xlabel('Gene $X_1$')
    ax.set_ylabel('Gene $X_2$')
    
    ax.legend()
    
    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap='rainbow', norm=norm)
    sm.set_array(all_g_values)
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label('Normalized predicted growth rate')
    
    # Format colorbar ticks
    cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{(x):.2f}'))
    
    # Save as PDF
    plt.savefig(os.path.join(exp_dir, 'g_values_plot.pdf'), format='pdf', bbox_inches='tight')
    
    plt.show()

# Plot with f_net and df
plot_g_values(df, f_net, dim_reducer=dim_reducer)

### Plot score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# Assume the following are defined:
# - df: DataFrame with columns 'samples', 'x1', 'x2', ..., 'xn'
# - dim: Number of dimensions (n)
# - device: torch.device (e.g., 'cpu' or 'cpu')
# - sf2m_score_model: Your model function that takes t_tensor and data_tensor

# Step 1: Extract all time points and data points


all_times = df['samples'].values
all_data = df[[f'x{i}' for i in range(1, dim + 1)]].values

# Step 2: Convert to PyTorch tensors
data_tensor = torch.tensor(all_data, dtype=torch.float32).to(device)
data_tensor.requires_grad_(True)  # Enable gradient tracking
t_tensor = torch.tensor(all_times, dtype=torch.float32).unsqueeze(1).to(device)

# Step 3: Compute log density values and gradients
log_density_values = sf2m_score_model(t_tensor, data_tensor)
log_density_values.backward(torch.ones_like(log_density_values))
gradients = data_tensor.grad

# Step 4: Move data to CPU for plotting
data_np = data_tensor.detach().cpu().numpy()
if dim_reducer is not None:
    data_np_2d=dim_reducer.transform(data_np)
else:
    data_np_2d=data_np[:,:2]
gradients_np = gradients.detach().cpu().numpy()
data_end = data_np + gradients_np
if dim_reducer is not None:
    data_end_2d=dim_reducer.transform(data_end)
else:
    data_end_2d=data_end[:,:2]
gradients_np=data_end_2d - data_np_2d

gradients_np = gradients_np / np.linalg.norm(gradients_np, axis = 1, keepdims=True) * 5
times_np = all_times
data_np = data_np_2d

In [ ]:
import anndata
import scvelo as scv
import scanpy as sc
import numpy as np

# Assume your data is already defined
# all_data: 50-dimensional data with shape (n_cells, 50)  
# gradients: 50-dimensional vector field with shape (n_cells, 50)
# pca: Trained PCA or UMAP model for generating X_umap
# X_umap: Pre-computed UMAP embeddings with shape (n_cells, 2)

# Create AnnData object
adata = anndata.AnnData(X=all_data)

# Set 'Ms' layer to avoid KeyError: 'Ms'
adata.layers['Ms'] = all_data  # Use original 50D data as state matrix
# Optional: normalize and log transform (based on data needs)
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)
# adata.layers['Ms'] = adata.X.copy()

# Set velocity vectors
adata.layers['velocity'] = gradients.detach().cpu().numpy()  # Store velocity vectors in layers
print(adata.layers['velocity'].shape)  # Confirm shape

# Set pre-computed UMAP embeddings
if dim_reducer is not None:
    X_umap = dim_reducer.transform(all_data)  # Assume pca is trained dim reduction model
else:
    X_umap = all_data[:,:2]
adata.obsm['X_umap'] = X_umap
print(adata.obsm['X_umap'].shape)  # Confirm UMAP embedding shape
adata.obs['time'] = all_times
if adata.layers['velocity'].shape[1] != 2:
    # Compute neighbor graph (required for velocity graph)
    sc.pp.neighbors(adata, n_neighbors=30, use_rep='X')  # Calculate neighbors based on high-dim data

    # Compute velocity graph
    scv.tl.velocity_graph(adata, vkey='velocity', n_jobs=16)  # Build velocity graph from high-dim velocity vectors

    # Project velocities to UMAP space
    scv.tl.velocity_embedding(adata, basis='umap', vkey='velocity')  # Project velocities to UMAP
else:
    adata.obsm['velocity_umap'] = adata.layers['velocity']

# Plot
adata.obs['time_categorical'] = pd.Categorical(adata.obs['time'])

# Visualization settings
scv.settings.set_figure_params('scvelo')  # Set scvelo plotting style

In [ ]:
scv.pl.velocity_embedding_stream(
    adata,
    basis='umap',
    color='time_categorical',
    figsize=(7, 5),
    density=3,
    title='Score Stream Plot',
    legend_loc='right',
    palette='viridis',
)

### Plot drift

In [ ]:
import numpy as np
import torch
device = 'cpu'
f_net.to(device)
sf2m_score_model.to(device)
# Assuming df, dim, device, f_net are defined
all_times = df['samples'].values
all_data = df[[f'x{i}' for i in range(1, dim + 1)]].values

# Convert to PyTorch tensors
t_tensor = torch.tensor(all_times, dtype=torch.float32).unsqueeze(1).to(device)
data_tensor = torch.tensor(all_data, dtype=torch.float32).to(device)

# Calculate gradients
with torch.no_grad():  # No need to track gradients to save memory
    gradients = f_net.v_net(t_tensor, data_tensor)

# Convert to NumPy array
gradients_np = gradients.cpu().numpy()


import numpy as np
import matplotlib.pyplot as plt
import torch

# Assume the following are defined:
# - df: DataFrame with columns 'samples', 'x1', 'x2', ..., 'xn'
# - dim: Number of dimensions (n)
# - device: torch.device (e.g., 'cpu' or 'cpu')
# - sf2m_score_model: Your model function that takes t_tensor and data_tensor

# Step 1: Extract all time points and data points
all_times = df['samples'].values
all_data = df[[f'x{i}' for i in range(1, dim + 1)]].values

#t_value = 4.0  
#t_tensor = torch.tensor([t_value] * all_data.shape[0]).unsqueeze(1).float().to(device)
t_tensor = torch.tensor(all_times, dtype=torch.float32).unsqueeze(1).to(device)

# Step 2: Convert to PyTorch tensors
data_tensor = torch.tensor(all_data, dtype=torch.float32).to(device)
data_tensor.requires_grad_(True)  # Enable gradient tracking
#t_tensor = torch.tensor(all_times, dtype=torch.float32).unsqueeze(1).to(device)

# Step 3: Compute log density values and gradients
log_density_values = sf2m_score_model(t_tensor, data_tensor)
log_density_values.backward(torch.ones_like(log_density_values))
gradients_score = data_tensor.grad

In [ ]:
overall=gradients_np+gradients_score.cpu().numpy()

In [ ]:
import anndata
import scvelo as scv
import scanpy as sc
import numpy as np
import pandas as pd
all_data = df[[f'x{i}' for i in range(1, dim + 1)]].values

# Assume data is defined: all_data, gradients, X_umap, all_times, cell_type
adata = anndata.AnnData(X=all_data)
adata.layers['Ms'] = all_data
adata.layers['velocity'] = overall
adata.obsm['X_umap'] = data_np
adata.obs['time'] = all_times
velocity_norm = adata.layers['velocity'] 
# Verify shapes
print("Velocity shape:", adata.layers['velocity'].shape)
print("UMAP shape:", adata.obsm['X_umap'].shape)

# Compute neighborhood graph
#sc.pp.neighbors(adata, n_neighbors=30, use_rep='X')

# Compute velocity graph
#scv.tl.velocity_graph(adata, vkey='velocity', n_jobs=1)
#
# Handle velocity projection
if adata.layers['velocity'].shape[1] != 2:
    sc.pp.neighbors(adata, n_neighbors=30, use_rep='X')
    scv.tl.velocity_graph(adata, vkey='velocity', n_jobs=16)
    scv.tl.velocity_embedding(adata, basis='umap', vkey='velocity')
else:
    adata.obsm['velocity_umap'] = adata.layers['velocity']

# Plot

adata.obs['time_categorical'] = pd.Categorical(adata.obs['time'])

In [ ]:
scv.settings.set_figure_params('scvelo')
scv.pl.velocity_embedding_stream(
    adata,
    basis='umap',
    color='time_categorical',
    figsize=(7, 5),
    density=1,
    title='All velocity Stream Plot',
    legend_loc='right',
    palette='viridis',
    save=exp_dir+'/all_velocity_stream_plot.svg'
)

### Generate trajectory

In [ ]:
from DeepRUOT.utils import euler_sdeint
import random
n_times = all_times.max() + 1
data=torch.tensor(df[df['samples']==0].values,dtype=torch.float32).requires_grad_()
data_t0 = data[:, 1:].to(device).requires_grad_()
print(data_t0.shape)
x0=data_t0.to(device)

dim_reducer = joblib.load(os.path.join(exp_dir, 'dim_reduction.pkl'))

class SDE(torch.nn.Module):
    noise_type = "diagonal"
    sde_type = "ito"

    def __init__(self, ode_drift, g, score, input_size=(3, 32, 32), sigma=1.0):
        super().__init__()
        self.drift = ode_drift
        self.score = score
        self.input_size = input_size
        self.sigma = sigma
        self.g_net = g

    # Drift
    def f(self, t, y):
        z, lnw = y
        drift=self.drift(t, z)
        dlnw = self.g_net(t, z)
        num = z.shape[0]
        t = t.expand(num, 1)  # Keep gradient information of t and expand its shape
        return (drift+self.score.compute_gradient(t, z), dlnw)

    # Diffusion
    def g(self, t, y):
        return torch.ones_like(y)*self.sigma
    
x0_subset = x0.to(device)

x0_subset = x0_subset.to(device)
lnw0 = torch.log(torch.ones(x0_subset.shape[0], 1) / x0_subset.shape[0]).to(device)
initial_state = (x0_subset, lnw0)

# Define SDE object
sde = SDE(f_net.v_net, 
          f_net.g_net, 
          sf2m_score_model, 
          input_size=(2,), 
          sigma=config['score_train']['sigma'])

# Define time points, assuming total 200 integration steps
ts = torch.linspace(0, n_times - 1, 100, device=device)

# Manual SDE integration
sde_traj, traj_lnw = euler_sdeint(sde, initial_state, dt=0.1, ts=ts)
# Transfer to CPU if needed:
sde_traj, traj_lnw = sde_traj.cpu(), traj_lnw.cpu()


sample_number = 100  # For example, sample 10
sample_indices = random.sample(range(sde_traj.size(1)), sample_number)
sampled_sde_trajec = sde_traj[:, sample_indices, :]
sampled_sde_trajec.shape
sampled_sde_trajec = sampled_sde_trajec.tolist()
sampled_sde_trajec = np.array(sampled_sde_trajec, dtype=object)
np.save(exp_dir+'/sde_trajec.npy', sampled_sde_trajec)


ts_points = torch.tensor(sorted(df.samples.unique()), dtype=torch.float32)
print(ts_points)

sde_point, traj_lnw = euler_sdeint(sde, initial_state, dt=0.1, ts=ts_points)
print(sde_point.shape)
print(traj_lnw.shape)
weight = torch.exp(traj_lnw)
weight_normed = weight/weight.sum(dim = 1, keepdim = True)

sde_point_np = sde_point.detach().cpu().numpy()
sde_point_list = sde_point_np.tolist()
sde_point_array = np.array(sde_point_list, dtype=object)
np.save(exp_dir+'/sde_point.npy', sde_point_array)
np.save(exp_dir+'/sde_weight.npy', weight_normed.detach().cpu().numpy())

from DeepRUOT.plots import new_plot_comparisions2
sde_point = np.load(exp_dir+'/sde_point.npy', allow_pickle=True)
sde_point.shape

sde_trajec = np.load(exp_dir+'/sde_trajec.npy', allow_pickle=True)
samples = df.iloc[:, 0]  # Get samples column
features = df.iloc[:, 1:]  # Get feature columns

# Transform data based on whether dim_reducer is provided
if dim_reducer is not None:
    # Transform original data
    reduced_features = dim_reducer.transform(features)
    
    # Transform generated points
    generated_flattened = sde_point.reshape(-1, features.shape[1])
    generated_reduced = dim_reducer.transform(generated_flattened)
    generated_reduced = generated_reduced.reshape(sde_point.shape[0], -1, 2)
    
    # Transform trajectories 
    trajectories_flattened = sde_trajec.reshape(-1, features.shape[1])
    trajectories_reduced = dim_reducer.transform(trajectories_flattened)
    trajectories_reduced = trajectories_reduced.reshape(sde_trajec.shape[0], -1, 2)
else:
    # Use first two dimensions
    reduced_features = features.iloc[:,:2].values
    generated_reduced = sde_point[:,:,:2] 
    trajectories_reduced = sde_trajec[:,:,:2]

# Create dataframe with reduced dimensions
new_df = pd.DataFrame(reduced_features, columns=['x1', 'x2'])
new_df['samples'] = samples

new_plot_comparisions2(
    new_df, generated_reduced, trajectories_reduced,
    palette='viridis', df_time_key='samples', 
    save=True, path=exp_dir, file='sde_trajectories.pdf',
    x='x1', y='x2', z='x3', is_3d=False
)

In [ ]:
celltype

In [ ]:
import scanpy as sc
meta_data = sc.read_h5ad('/lustre/home/2100011778/DeepRUOTv2_test_data/test_data/WO_FG_D10_D20_SCUD_tumor_cca_downsample_0120_20dims.h5ad')
celltype = meta_data.obs['celltype']

data=torch.tensor(df[df['samples'] == 0].values,dtype=torch.float32).requires_grad_()
data_t0 = data[:, 1:].to(device).requires_grad_()
print(data_t0.shape)
x0=data_t0.to(device)

In [ ]:
meta_data['celltype']

In [ ]:
# 读取 CSV 文件
df_new = pd.read_csv(os.path.join(DATA_DIR, config['data']['file_path']))
all_labels = meta_data.obs['celltype'].values

# 添加 Annotation 列
df_new['Annotation'] = all_labels
df_new
# # 获取 Annotation 的类别和颜色
# if pd.api.types.is_categorical_dtype(adata.obs['Annotation']):
#     categories = adata.obs['Annotation'].cat.categories
# else:
#     categories = adata.obs['Annotation'].unique()  # 如果不是 categorical 类型

# df_new.to_csv(os.path.join(DATA_DIR, 'test_data_0710_pro_HB_with_annotation.csv'), index=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# 假设数据已经加载到 DataFrame 中
# df = pd.read_csv('your_data.csv')  # 替换为你的数据文件路径

# 步骤 1：准备数据
# 提取特征和标签
X = df_new.iloc[:,:-1].values
y = df_new['Annotation'].values

# 对标签进行编码
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(y_encoded[:100])
# 划分训练集和测试集（80% 训练，20% 测试）
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 转换为 PyTorch 张量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# 步骤 2：构建 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 第一隐藏层
        self.relu = nn.LeakyReLU()                          # 激活函数
        self.fc2 = nn.Linear(hidden_size, hidden_size) # 第二隐藏层
        self.fc3 = nn.Linear(hidden_size, num_classes) # 输出层
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

# 设置模型参数
input_size = 21  # 特征数量
hidden_size = 128 # 隐藏层神经元数量
num_classes = len(label_encoder.classes_)  # 类别数量
model = MLP(input_size, hidden_size, num_classes)

# 步骤 3：训练模型
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 优化器

In [ ]:
# 训练循环
num_epochs = 10000
model = model.cuda()
X_train = X_train.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 优化器
for epoch in range(num_epochs):
    model.train()  # 设置为训练模式
    outputs = model(X_train)
    loss = criterion(outputs, y_train.cuda())
    
    optimizer.zero_grad()  # 清空梯度
    loss.backward()        # 反向传播
    optimizer.step()       # 更新参数
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        # 步骤 4：评估模型
        model.eval()  # 设置为评估模式
        with torch.no_grad():
            outputs = model(X_test.cuda())
            _, predicted = torch.max(outputs, 1)  # 获取预测类别
            accuracy = accuracy_score(y_test, predicted.cpu())
            print(f'测试集准确率: {accuracy:.4f}')

In [ ]:
torch.save(model.state_dict(), exp_dir + '/mlp_classifier.pth')
print("模型已保存到 'mlp_classifier.pth'")
model.eval()

# 初始化存储分类结果和颜色的列表
predicted_labels_list = []
# predicted_colors_list = []

ts = [0.0, 1.0, 2.0, 3.0]
# 对每个时间点进行分类
predicted_labels_list.append(df_new[df_new['samples']==0]['Annotation'].values)

for i in range(1, len(sde_point)):
    t = ts[i]  # 当前时间点
    traj_t = np.array(sde_point[i], dtype = np.float64)  # 当前时间点的特征，形状 (n, 12)
    traj_t = torch.tensor(traj_t)
    n_samples = traj_t.shape[0]
    
    # 构造输入：samples (时间) + 特征
    samples_t = t * torch.ones((n_samples, 1))  # (n, 1)
    input_t = torch.cat((samples_t, traj_t), dim=1)  # (n, 13)
    
    # 使用模型预测
    with torch.no_grad():
        outputs = model(input_t.float().cuda())
        _, predicted = torch.max(outputs, 1)  # 预测类别索引
        predicted_labels = label_encoder.inverse_transform(predicted.detach().cpu().numpy())  # 转换为原始标签
    
    # 获取颜色
    # predicted_colors = [label_to_color[label] for label in predicted_labels]
    
    # 存储结果
    predicted_labels_list.append(predicted_labels)
    # predicted_colors_list.append(predicted_colors)




In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import importlib
import nbformat
importlib.reload(nbformat)



# 第一步：统计细胞从一个时间点到下一个时间点的转变
links = []
num_timepoints = len(predicted_labels_list)

for t in range(num_timepoints - 1):
    # 使用 pandas 快速统计
    df = pd.DataFrame({
        'source': predicted_labels_list[t],
        'target': predicted_labels_list[t+1]
    })
    
    # 统计每种转变的数量
    counts = df.groupby(['source', 'target']).size().reset_index(name='value')
    
    # 为节点名称添加时间后缀，以区別不同时间的节点
    counts['source'] = counts['source'].astype(str) + f'_T{t+1}'
    counts['target'] = counts['target'].astype(str) + f'_T{t+2}'
    
    links.append(counts)

# 将所有时间点的连接合并成一个 DataFrame
all_links_df = pd.concat(links, axis=0)


# 第二步：构建绘图所需的数据结构
# 获取所有不重复的节点
all_nodes = pd.unique(all_links_df[['source', 'target']].values.ravel('K'))

# 创建一个从节点名称到索引的映射
node_indices = {node: i for i, node in enumerate(all_nodes)}

# 将 DataFrame 中的 source 和 target 替换为对应的索引
all_links_df['source_idx'] = all_links_df['source'].map(node_indices)
all_links_df['target_idx'] = all_links_df['target'].map(node_indices)

# 为不同细胞类型定义颜色
# 提取基础细胞类型名（去掉_T*后缀）
base_types = sorted(list(set([node.split('_')[0] for node in all_nodes])))
color_palette = ['#440154', '#3b528b', '#21908d', '#5dc863', '#fde725'] # 可自定义颜色
color_map = {base_type: color_palette[i % len(color_palette)] for i, base_type in enumerate(base_types)}
node_colors = [color_map[node.split('_')[0]] for node in all_nodes]


# 第三步：使用 Plotly 绘图
fig = go.Figure(data=[go.Sankey(
    # --- 节点定义 ---
    node=dict(
      pad=25,
      thickness=20,
      line=dict(color="black", width=0.5),
      label=all_nodes,
      color=node_colors,
      # 核心：手动指定 x 坐标来按时间列对齐
      x=[int(node.split('_T')[1]) / (num_timepoints + 1) for node in all_nodes],
      # y 坐标可由plotly自动确定，也可手动微调
      # y=[...] 
    ),
    # --- 连接定义 ---
    link=dict(
      source=all_links_df['source_idx'],
      target=all_links_df['target_idx'],
      value=all_links_df['value'],
      # 可以为连接也设置颜色
      # color = ...
  ))])

# 更新图表标题和布局
fig.update_layout(
    title_text="细胞谱系演变桑基图",
    font_family="Arial",
    font_size=12,
    plot_bgcolor='white'
)

fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --------------------


# ####################################################################
# ### 主要改动：设定目标细胞类型并筛选数据 ###
# ####################################################################

target_types = ['SCUD']

# 1. 对每个时间点，检查哪些细胞的类型在 `target_types` 列表中
#    np.isin 会返回一个布尔数组 (True/False mask)
masks = [np.isin(labels, target_types) for labels in predicted_labels_list]

# 2. 合并所有时间点的掩码。只要一个细胞在任何时间点出现过目标类型，
#    我们就保留它。我们使用逻辑“或”操作来合并。
#    np.logical_or.reduce 会对列表中的所有数组进行逐元素的“或”运算。
combined_mask = np.logical_or.reduce(masks)

# 3. 根据最终的合并掩码，获取所有相关细胞的索引
target_indices = np.where(combined_mask)[0]

# 4. 使用这些索引筛选出完整的谱系数据
if len(target_indices) > 0:
    filtered_predicted_label_list = [labels[target_indices] for labels in predicted_labels_list]
else:
    filtered_predicted_label_list = [np.array([]) for _ in predicted_labels_list]
    print(f"警告：在任何时间点都未找到细胞类型在 {target_types} 中的谱系。将生成一个空图。")
    
# ####################################################################
# ### 后续代码与之前完全相同 ###
# ####################################################################

# 第一步：统计转变
links = []
num_timepoints = len(filtered_predicted_label_list)
if num_timepoints > 1 and len(filtered_predicted_label_list[0]) > 0:
    for t in range(num_timepoints - 1):
        df = pd.DataFrame({'source': filtered_predicted_label_list[t], 'target': filtered_predicted_label_list[t+1]})
        counts = df.groupby(['source', 'target']).size().reset_index(name='value')
        counts['source'] = counts['source'].astype(str) + f'_T{t+1}'
        counts['target'] = counts['target'].astype(str) + f'_T{t+2}'
        links.append(counts)
    all_links_df = pd.concat(links, axis=0)

    # 第二步：构建绘图数据
    all_nodes = pd.unique(all_links_df[['source', 'target']].values.ravel('K'))
    node_indices = {node: i for i, node in enumerate(all_nodes)}
    all_links_df['source_idx'] = all_links_df['source'].map(node_indices)
    all_links_df['target_idx'] = all_links_df['target'].map(node_indices)
    base_types = sorted(list(set([node.split('_')[0] for node in all_nodes])))
    color_palette = ['#440154', '#3b528b', '#21908d', '#5dc863', '#fde725']
    color_map = {base_type: color_palette[i % len(color_palette)] for i, base_type in enumerate(base_types)}
    node_colors = [color_map[node.split('_')[0]] for node in all_nodes]

    # 第三步：绘图
    fig = go.Figure(data=[go.Sankey(
        node=dict(
          pad=25, thickness=20, line=dict(color="black", width=0.5),
          label=all_nodes, color=node_colors,
          x=[int(node.split('_T')[1]) / (num_timepoints + 1) for node in all_nodes],
        ),
        link=dict(
          source=all_links_df['source_idx'], target=all_links_df['target_idx'],
          value=all_links_df['value'],
      ))])
    fig.update_layout(title_text=f"包含 {target_types} 的细胞谱系图", font_family="Arial", font_size=12, plot_bgcolor='white')
else:
    fig = go.Figure()
    fig.update_layout(title_text=f"未找到包含 {target_types} 的细胞谱系", annotations=[dict(text="无数据显示", xref="paper", yref="paper", showarrow=False, font=dict(size=20))])

fig.show()